In [1]:
!pip install neo4j

     |████████████████████████████████| 69 kB 5.1 MB/s eta 0:00:011
     |████████████████████████████████| 509 kB 8.6 MB/s eta 0:00:01
  Created wheel for neo4j: filename=neo4j-4.2.1-py3-none-any.whl size=95192 sha256=d0bc4327445b94f7cf9115d46dc0234490d7ec6af723c74394b1f323f3ed9782
  Stored in directory: /root/.cache/pip/wheels/c3/b2/39/328111acf2c9619133fe4d156e1d9e85a7503fe5a55276cbd9
Successfully built neo4j
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [14]:
import os
from neo4j import GraphDatabase
neo4jauth = tuple(os.environ["NEO4J_AUTH"].split("/"))

In [15]:
uri = "neo4j://neo4j:7687"
driver = GraphDatabase.driver(uri, auth=(neo4jauth))

In [17]:
def create_friend_of(tx, name, friend):
    tx.run("CREATE (a:Person)-[:KNOWS]->(f:Person {name: $friend})\
           WHERE a.name = $name \
           RETURN f.name AS friend", name=name, friend=friend)

with driver.session() as session:
    session.write_transaction(create_friend_of, "Alice", "Bob")

with driver.session() as session:
    session.write_transaction(create_friend_of, "Alice", "Carl")

driver.close()

CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'WHERE': expected 
  <EOF>
  "RETURN"
  "CREATE"
  "DELETE"
  "SET"
  "REMOVE"
  "DETACH"
  "MATCH"
  "WITH"
  "UNWIND"
  "USE"
  "CALL"
  "LOAD"
  "FROM"
  "FOREACH"
  "MERGE"
  "OPTIONAL"
  "UNION"
  "," (line 1, column 66 (offset: 65))
"CREATE (a:Person)-[:KNOWS]->(f:Person {name: $friend})           WHERE a.name = $name            RETURN f.name AS friend"
                                                                  ^}

In [ ]:
def get_friends_of(tx, name):
    friends = []
    result = tx.run("MATCH (a:Person)-[:KNOWS]->(f) "
                         "WHERE a.name = $name "
                         "RETURN f.name AS friend", name=name):
    for record in result:
        friends.append(record["friend"])
    return friends

with driver.session() as session:
    friends = session.read_transaction(get_friends_of, "Alice")
    for friend in friends:
        print(friend)

driver.close()

In [ ]:
CREATE (a:Person)-[:KNOWS]->(f:Person {name: "bob"}) WHERE a.name = alice RETURN f.name AS friend